# Deep Learning with CICIDS Dataset 

In [2]:
!pip install tensorflow

     -------------------------------------- 455.9/455.9 MB 7.7 MB/s eta 0:00:00
     ------------------------------------- 895.9/895.9 kB 11.4 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 13.5 MB/s eta 0:00:00
     ---------------------------------------- 5.9/5.9 MB 14.4 MB/s eta 0:00:00
     ---------------------------------------- 42.6/42.6 kB ? eta 0:00:00
     -------------------------------------- 124.6/124.6 kB 7.6 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB ? eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB ? eta 0:00:00
     --------------------------------------- 14.2/14.2 MB 17.7 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 13.4 MB/s eta 0:00:00
     ------------------------------------- 438.7/438.7 kB 13.8 MB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 18.0 MB/s eta 0:00:00
     ------------------------------------- 781.3/781.3 kB 16.4 MB

In [3]:
import os
from os.path import join
import glob
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow.keras as keras

In [4]:
keras.__version__

'2.10.0'

In [5]:
%tensorflow_version

UsageError: Line magic function `%tensorflow_version` not found.


In [6]:
def display_metrics(y_test, y_pred, label_names):
  print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))

  print('Micro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='micro')))
  print('Micro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='micro')))
  print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')))

  print('Macro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='macro')))
  print('Macro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='macro')))
  print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')))

  print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
  print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
  print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

  print('\nClassification Report\n')
  print(classification_report(y_test, y_pred, target_names=label_names))

In [7]:
def display_all(df):
    with pd.option_context("display.max_rows", 100, "display.max_columns", 100): 
        print(df)

In [8]:
def make_value2index(attacks):
    #make dictionary
    attacks = sorted(attacks)
    d = {}
    counter=0
    for attack in attacks:
        d[attack] = counter
        counter+=1
    return d

In [9]:
# chganges label from string to integer/index
def encode_label(Y_str):
    labels_d = make_value2index(np.unique(Y_str))
    Y = [labels_d[y_str] for y_str  in Y_str]
    Y = np.array(Y)
    return np.array(Y)

In [10]:
# All columns
col_names = np.array(['dst sport count', 'src dport count', 'dst src count', 'dport count', 'sport count', 'dst host count','src host count','Source Port', 'Destination Port',
                      'Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets',
                      'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean',
                      'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std',
                      'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total',
                      'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
                      'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Header Length', 'Bwd Header Length',
                      'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
                      'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
                      'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Avg Fwd Segment Size',
                      'Avg Bwd Segment Size','Subflow Fwd Packets', 'Subflow Fwd Bytes',
                      'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward',
                      'act_data_pkt_fwd', 'min_seg_size_forward', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
                      'Idle Std', 'Idle Max', 'Idle Min', 'Label'])

In [11]:
df_train = pd.read_csv('train_MachineLearningCVE.csv',names=col_names, skiprows=1)  

In [12]:
print('Train set size: ', df_train.shape)

Train set size:  (2264594, 79)


In [13]:
df_test = pd.read_csv('test_MachineLearningCVE.csv',names=col_names, skiprows=1)  
print('Test set size: ', df_test.shape)

Test set size:  (566149, 79)


In [14]:
df_train.head()

,dst sport count,src dport count,dst src count,dport count,sport count,dst host count,src host count,Source Port,Destination Port,Protocol,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,6018089.0,5,3,177,994,159,0,35.400000,69.143329,...,20.0,275923.0000,0.000,275923,275923,5742166.0,0.000000,5742166,5742166,0
1,443,323049.0,8,6,531,3208,194,0,66.375000,85.880211,...,20.0,0.0000,0.000,0,0,0.0,0.000000,0,0,0
2,80,39270118.0,9,10,898,3944,431,0,99.777778,187.795619,...,20.0,123253.6667,149943.928,296394,36667,10000000.0,300.561142,10000000,9999980,0
3,4848,43.0,1,1,2,6,2,2,2.000000,0.000000,...,24.0,0.0000,0.000,0,0,0.0,0.000000,0,0,10
4,80,5754816.0,3,1,12,0,6,0,4.000000,3.464102,...,20.0,0.0000,0.000,0,0,0.0,0.000000,0,0,0


In [15]:
df_train['Label'].value_counts()

0     1818477
4      184858
10     127144
2      102421
3        8234
7        6350
11       4718
6        4637
5        4399
1        1573
12       1206
14        522
9          29
13         17
8           9
Name: Label, dtype: int64

In [16]:
df_train.describe()

,dst sport count,src dport count,dst src count,dport count,sport count,dst host count,src host count,Source Port,Destination Port,Protocol,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
count,2.264594e+06,2.264594e+06,2.264594e+06,2.264594e+06,2.264594e+06,2.264594e+06,2.264594e+06,2.264594e+06,2.264594e+06,2.264594e+06,...,2.264594e+06,2.264594e+06,2.264594e+06,2.264594e+06,2.264594e+06,2.264594e+06,2.264594e+06,2.264594e+06,2.264594e+06,2.264594e+06
mean,8.083469e+03,1.479315e+07,9.843101e+00,1.103084e+01,5.496153e+02,1.764019e+04,2.077673e+02,1.869240e+01,5.822619e+01,6.899965e+01,...,2.611347e+01,8.127506e+04,4.110607e+04,1.528082e+05,5.799099e+04,8.322365e+06,5.050259e+05,8.702922e+06,7.925571e+06,1.064424e+00
std,1.829501e+04,3.366280e+07,8.089885e+02,1.076331e+03,1.053290e+04,2.448270e+06,7.185458e+02,6.026335e+01,1.863955e+02,2.819866e+02,...,6.577371e+00,6.331009e+05,3.907903e+05,1.014727e+06,5.600375e+05,2.363823e+07,4.610524e+06,2.437702e+07,2.337079e+07,2.576436e+00
min,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.300000e+01,1.550000e+02,2.000000e+00,1.000000e+00,1.200000e+01,0.000000e+00,6.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,...,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,8.000000e+01,3.131600e+04,2.000000e+00,2.000000e+00,6.200000e+01,1.220000e+02,3.700000e+01,2.000000e+00,3.400000e+01,0.000000e+00,...,2.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,4.430000e+02,3.211786e+06,5.000000e+00,4.000000e+00,1.870000e+02,4.820000e+02,8.100000e+01,3.600000e+01,5.000000e+01,2.616295e+01,...,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,6.553500e+04,1.200000e+08,2.197590e+05,2.919220e+05,1.290000e+07,6.554530e+08,2.482000e+04,2.325000e+03,5.940857e+03,7.049469e+03,...,1.380000e+02,1.100000e+08,7.420000e+07,1.100000e+08,1.100000e+08,1.200000e+08,7.690000e+07,1.200000e+08,1.200000e+08,1.400000e+01


In [17]:
print('Test set: ')
df_test['Label'].value_counts()

Test set: 


0     454620
4      46215
10     31786
2      25606
3       2059
7       1588
11      1179
6       1159
5       1100
1        393
12       301
14       130
9          7
13         4
8          2
Name: Label, dtype: int64

In [18]:
df_test.describe()

,dst sport count,src dport count,dst src count,dport count,sport count,dst host count,src host count,Source Port,Destination Port,Protocol,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
count,566149.000000,5.661490e+05,566149.000000,566149.000000,5.661490e+05,5.661490e+05,566149.000000,566149.000000,566149.000000,566149.000000,...,566149.000000,5.661490e+05,5.661490e+05,5.661490e+05,5.661490e+05,5.661490e+05,5.661490e+05,5.661490e+05,5.661490e+05,566149.000000
mean,8023.536525,1.475572e+07,7.433398,7.845504,5.480510e+02,1.025244e+04,206.930559,18.798663,58.104949,68.552052,...,26.122669,8.265636e+04,4.124628e+04,1.546799e+05,5.951514e+04,8.290722e+06,4.991163e+05,8.667073e+06,7.897870e+06,1.064391
std,18237.981102,3.361749e+07,438.398407,583.054409,7.455954e+03,1.277379e+06,711.715142,60.642342,184.868811,277.965989,...,6.579084,7.072071e+05,4.035803e+05,1.069064e+06,6.407873e+05,2.359744e+07,4.572702e+06,2.432631e+07,2.333391e+07,2.576357
min,0.000000,0.000000e+00,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,53.000000,1.550000e+02,2.000000,1.000000,1.200000e+01,2.000000e+00,6.000000,0.000000,6.000000,0.000000,...,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
50%,80.000000,3.131600e+04,2.000000,2.000000,6.200000e+01,1.240000e+02,37.000000,2.000000,34.000000,0.000000,...,24.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
75%,443.000000,3.175237e+06,5.000000,4.000000,1.870000e+02,4.800000e+02,81.000000,36.000000,50.000000,26.162951,...,32.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
max,65533.000000,1.200000e+08,199057.000000,269619.000000,2.866110e+06,5.840000e+08,24820.000000,1983.000000,4638.923469,7125.596846,...,126.000000,1.070000e+08,6.390000e+07,1.070000e+08,1.070000e+08,1.200000e+08,7.514502e+07,1.200000e+08,1.200000e+08,14.000000


In [19]:
df_label = df_train['Label']
data = df_train.drop(columns=['Label'])
Xtrain = data.values
y_train = encode_label(df_label.values)

In [20]:
df_label = df_test['Label']
data = df_test.drop(columns=['Label'])
Xtest = data.values
y_test = encode_label(df_label.values)

In [21]:
from sklearn.preprocessing import MinMaxScaler

In [22]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(Xtrain)
X_train

array([[1.22072175e-03, 5.01507425e-02, 1.82018402e-05, ...,
        0.00000000e+00, 4.78513833e-02, 4.78513833e-02],
       [6.75974670e-03, 2.69207504e-03, 3.18532204e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.22072175e-03, 3.27250989e-01, 3.64036804e-05, ...,
        3.90846738e-06, 8.33333333e-02, 8.33331667e-02],
       ...,
       [1.14290074e-02, 3.58333339e-07, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.71915770e-02, 6.50000011e-07, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.97756924e-01, 6.32742594e-02, 0.00000000e+00, ...,
        0.00000000e+00, 6.30086083e-02, 6.30086083e-02]])

In [23]:
X_test = scaler.transform(Xtest)
X_test

array([[8.08728161e-04, 1.50000003e-06, 4.55046005e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.22072175e-03, 7.01777962e-01, 4.09541405e-05, ...,
        0.00000000e+00, 6.96666667e-01, 6.96666667e-01],
       [1.34279393e-03, 1.30000002e-06, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.22072175e-03, 1.16711919e-02, 9.10092010e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.08728161e-04, 6.83175011e-04, 1.36513802e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.54467079e-02, 6.00000010e-07, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [24]:
from sklearn.preprocessing import StandardScaler

In [25]:
scaler = StandardScaler()

X_train = scaler.fit_transform(Xtrain)
X_test = scaler.transform(Xtest)

X_test

array([[-4.38943193e-01, -4.39445625e-01, -9.69494948e-03, ...,
        -1.09537648e-01, -3.57013384e-01, -3.39122989e-01],
       [-4.37467380e-01,  2.06222335e+00,  1.93944960e-04, ...,
        -1.09537648e-01,  3.07244625e+00,  3.23799222e+00],
       [-4.37030103e-01, -4.39446338e-01, -1.09310613e-02, ...,
        -1.09537648e-01, -3.57013384e-01, -3.39122989e-01],
       ...,
       [-4.37467380e-01, -3.97845903e-01, -8.45883767e-03, ...,
        -1.09537648e-01, -3.57013384e-01, -3.39122989e-01],
       [-4.38943193e-01, -4.37015613e-01, -7.22272587e-03, ...,
        -1.09537648e-01, -3.57013384e-01, -3.39122989e-01],
       [-3.14865627e-01, -4.39448833e-01, -1.09310613e-02, ...,
        -1.09537648e-01, -3.57013384e-01, -3.39122989e-01]])

In [26]:
from tensorflow.keras.utils import to_categorical

In [27]:
y_train_ada = y_train
y_test_ada = y_test

In [28]:
y_train = to_categorical(y_train, 15)
y_test = to_categorical(y_test, 15)

In [29]:
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
from sklearn.ensemble import RandomForestClassifier

#importing confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn import metrics
from sklearn.metrics import accuracy_score

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error,mean_absolute_error

Instructions for updating:
non-resource variables are not supported in the long term


In [30]:
labels_d = make_value2index(df_test['Label'])

In [31]:
print(labels_d)

{0: 454619, 1: 455012, 2: 480618, 3: 482677, 4: 528892, 5: 529992, 6: 531151, 7: 532739, 8: 532741, 9: 532748, 10: 564534, 11: 565713, 12: 566014, 13: 566018, 14: 566148}


In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, Flatten, Dense, Activation,Dropout,MaxPooling1D
from tensorflow.keras.constraints import max_norm

In [33]:
#hyper-params
batch_size = 7500 # increasing batch size with more gpu added
input_dim = X_train.shape[1]
num_class = 15                   # 15 intrusion classes, including benign traffic class
num_epochs = 6
learning_rates = 1e-4
regularizations = 1e-3
optim = tf.keras.optimizers.Adam(lr=learning_rates, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

print(input_dim)
print(num_class)

78
15


C:\Users\shara\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [34]:
X_train.shape

(2264594, 78)

In [35]:
#X_train_r = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train_r = np.zeros((len(X_train), input_dim, 1))
X_train_r[:, :, 0] = X_train[:, :input_dim]
print(X_train_r.shape)

(2264594, 78, 1)


In [36]:
X_test_r = np.zeros((len(X_test), input_dim, 1))
X_test_r[:, :, 0] = X_test[:, :input_dim]
print(X_test_r.shape)

(566149, 78, 1)


In [37]:
type(X_train_r)

numpy.ndarray

In [38]:
model = Sequential()

# input layer
model.add(Conv1D(filters=32, kernel_size=23, activation='relu', padding='same', kernel_initializer='he_uniform', input_shape=(78,1)))
model.add(Conv1D(filters=32, kernel_size=17, activation='relu', padding='same', kernel_initializer='he_uniform'))
model.add(MaxPooling1D(pool_size=2,strides=2))
model.add(Dropout(0.2))
model.add(BatchNormalization(axis=1))

model.add(Conv1D(filters=64, kernel_size=23, activation='relu', padding='same', kernel_initializer='he_uniform'))
model.add(Conv1D(filters=64, kernel_size=17, activation='relu', padding='same', kernel_initializer='he_uniform'))
model.add(MaxPooling1D(pool_size=2,strides=2))
model.add(Dropout(0.2))
model.add(BatchNormalization(axis=1)) 

model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_class))
model.add(Activation('softmax'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 78, 32)            768       
                                                                 
 conv1d_1 (Conv1D)           (None, 78, 32)            17440     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 39, 32)           0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 39, 32)            0         
                                                                 
 batch_normalization (BatchN  (None, 39, 32)           156       
 ormalization)                                                   
                                                  

In [39]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      #tf.keras.metrics.AUC(name='auc'),
]

metrics=METRICS

In [ ]:
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
import tensorflow.keras
import time
time_start = time.time()

nadam = Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
model.compile(loss = "categorical_crossentropy",optimizer = "nadam", metrics = metrics)

#model.compile(
#      optimizer=tf.keras.optimizers.Adam(lr=1e-4),
#      loss=tf.keras.losses.BinaryCrossentropy(),
#      metrics=metrics)
  

history = model.fit(X_train_r, y_train, 
                    epochs=1, 
                    batch_size=batch_size, 
                    verbose=2)
time_end = time.time()
train_time = time_end - time_start
print("train_time:",train_time)

Train on 2264594 samples
